# Calibrate cameras, create initial orthoimage

In [ ]:
import os
from glob import glob
import subprocess
import numpy as np
from tqdm import tqdm
import pandas as pd
import cv2
import geopandas as gpd
from shapely.geometry import Point
import rioxarray as rxr
import xarray as xr
import matplotlib.pyplot as plt
from ast import literal_eval
import rasterio
import warnings
warnings.filterwarnings('ignore')

# Locate image files
data_folder = '/Users/rdcrlrka/Research/Soo_locks'
image_folder = os.path.join(data_folder, 'inputs', 'original_images')
image_files = sorted(glob(os.path.join(image_folder, '*.tiff')))
print(f"{len(image_files)} images located")

# Grab other input files
gcp_folder = os.path.join(data_folder, 'inputs', 'gcp_new')
refdem_file = os.path.join(data_folder, 'inputs', 'lidar_DSM_filled_cropped.tif')

# Define output folders
out_folder = os.path.join(data_folder, 'camera_calibration')
os.makedirs(out_folder, exist_ok=True)
single_band_folder = os.path.join(out_folder, 'single_band_images')
calib_folder = os.path.join(out_folder, 'calibration_params')

## Merge GCP

In [ ]:
gcp_merged_file = os.path.join(gcp_folder, '..', 'GCP_merged.gpkg')
if not os.path.exists(gcp_merged_file):
    gdf_list = []
    gcp_files = sorted(glob(os.path.join(gcp_folder, '*.gcp')))
    for f in gcp_files:
        df = pd.read_csv(
            f, 
            skiprows=[0], 
            header=None,
            names = ['pt_number', 'lat', 'lon', 'Z', 'std_X', 'std_Y', 'std_Z', 'image_name', 'col_sample', 'row_sample', 'use_X', 'use_Y']
            )
        df['geometry'] = [Point(x,y) for (x,y) in df[['lon', 'lat']].values]
        gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")
        gdf = gdf.to_crs("EPSG:32619")
        
        gdf['X'] = [x.coords.xy[0][0] for x in gdf['geometry']]
        gdf['Y'] = [x.coords.xy[1][0] for x in gdf['geometry']]

        gdf_list += [gdf]
    df_merged = pd.concat(gdf_list).reset_index(drop=True)
    df_merged['channel'] = ['ch' + os.path.basename(x).split('ch')[1][0:2] for x in df_merged['image_name']]
    df_merged = df_merged[['channel', 'X', 'Y', 'Z', 'col_sample', 'row_sample', 'geometry']]

    gcp_merged = gpd.GeoDataFrame(df_merged, geometry='geometry', crs="EPSG:32619")
    gcp_merged.plot(kind='scatter', x='X', y='Y')
    plt.show()
    gcp_merged.to_file(gcp_merged_file, index=False)
    print(f'Merged GCP saved to:\n{gcp_merged_file}')

else:
    print('Merged GCP already exists in file, skipping.')
    

## Convert images to single band in case they're RGB

A couple IR images (near the windows) were captured in RGB 

In [ ]:
os.makedirs(single_band_folder, exist_ok=True)

# iterate over images
print('Saving single-band images to:', single_band_folder)
for image_file in tqdm(image_files):
    # convert images to single band
    out_fn = os.path.join(single_band_folder, os.path.basename(image_file))
    if os.path.exists(out_fn):
        continue
    cmd = [
        "gdal_translate",
        "-b", "1",
        image_file, out_fn
    ]
    subprocess.run(cmd)

## Calibrate cameras using GCP

In [ ]:
def calibrate_cameras(
        image_files: str = None, 
        gcp_file: str = None, 
        plot_results: bool = True,
        ):
    
    # --- Compile image and object points ---
    # Load merged GCP file
    gcp = gpd.read_file(gcp_file)
    # Remove erroneous GCP
    gcp = gcp.loc[gcp['Z'] < 0].reset_index(drop=True)
    
    # Compile image and object points
    image_points_list = []
    object_points_list = []
    image_size = None
    for image_file in image_files:
        ch = 'ch' + os.path.basename(image_file).split('ch')[1][0:2]
        gcp_image = gcp.loc[gcp['channel']==ch]

        obj = gcp_image[['X','Y','Z']].values.astype(np.float64)
        img = gcp_image[['col_sample','row_sample']].values.astype(np.float64)

        # reshape for fisheye: (1, N, 3)
        object_points_list.append(obj.reshape(1, -1, 3))
        image_points_list.append(img.reshape(1, -1, 2))

        if image_size is None:
            im = cv2.imread(image_file, 0)
            image_size = (im.shape[1], im.shape[0])

    if len(object_points_list) == 0:
        raise ValueError("No valid images for calibration")

    # --- Mean-center all object points for more stable calculations ---
    object_points_list = [x for x in object_points_list]

    # --- Initialize intrinsics & solving criteria --- 
    K_init = np.zeros((3, 3))
    D_init = np.zeros((4, 1))
    rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(len(image_files))]
    tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(len(image_files))]
    calibration_flags = cv2.fisheye.CALIB_FIX_SKEW + cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC # + cv2.fisheye.CALIB_CHECK_COND
    termination_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)

    # --- Calibrate camera intrinsics ---
    # Note: rotation/translation vectors seem unstable at this stage. 
    # Use solvePnP for pose estimation instead.
    print('Calibrating shared camera intrinsics...')
    rms, K, D, rvecs, tvecs = cv2.fisheye.calibrate(
        object_points_list,
        image_points_list,
        image_size,
        K_init,
        D_init,
        rvecs,
        tvecs,
        calibration_flags,
        termination_criteria
    )

    print("Shared calibration done")

    # --- Calculate camera matrix for full FOV ---
    K_full = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(K, D, image_size, None, balance=1)

    # --- Save camera extrinsics ---
    results = [] # initialize list of camera specs
    for i, image_file in enumerate(tqdm(image_files, desc='Extracting camera extrinsics')):
        ch = 'ch' + os.path.basename(image_file).split('ch')[1][0:2]
        r = {
            'image_file': image_file,
            'channel': ch,
            'K': K.tolist(),
            'dist': D.tolist(),
            'K_full': K_full.tolist(),
            'rvec': rvecs[i].tolist(),
            'tvec': tvecs[i].tolist(),
            }
        results += [r]

        # Plot results
        if plot_results:
            # Create undistorted image with full FOV
            image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
            map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K_full, image_size, cv2.CV_16SC2)
            image_undistorted = cv2.remap(image, map1, map2, interpolation=cv2.INTER_LINEAR)
            # fill nodata values with NaNs
            mask = np.ones(image.shape, dtype=np.uint8) * 255
            mask_undistorted = cv2.remap(mask, map1, map2, interpolation=cv2.INTER_NEAREST)
            valid_mask = mask_undistorted > 0
            image_undistorted_nodata = image_undistorted.astype(np.float32)
            image_undistorted_nodata[~valid_mask] = np.nan
            
            # Undistort GCP pixel coordinates
            image_pts = gcp_image[['col_sample','row_sample']].values.astype(np.float32).reshape(1,-1,2)
            undistorted_pts = cv2.fisheye.undistortPoints(image_pts, K, D, None, K_full).reshape(-1, 2)
            gcp_image['col_sample_undistorted'] = undistorted_pts[:, 0]
            gcp_image['row_sample_undistorted'] = undistorted_pts[:, 1]

            # Plot
            fig, ax = plt.subplots(1, 2, figsize=(10,5))
            ax[0].imshow(image, cmap='Grays_r')
            ax[0].plot(gcp_image['col_sample'], gcp_image['row_sample'], 'xr')
            ax[1].imshow(image_undistorted, cmap='Grays_r')
            ax[1].plot(gcp_image['col_sample_undistorted'], gcp_image['row_sample_undistorted'], 'xr')
            fig.suptitle(os.path.basename(image_file))
            for a in ax:
                a.set_xticks([])
                a.set_yticks([])
            fig.tight_layout()
            plt.show()

    # Compile results in dataframe 
    results_df = pd.DataFrame(results)

    return results_df


def orthorectify(
        image_file: str,
        dem_file: str,
        K: np.ndarray,
        D: np.ndarray,
        rvec: np.ndarray,
        tvec: np.ndarray,
        target_res: float = 0.002,
        max_elevation_above_camera: float = 0.0,
        fov_deg: float = 120.0,
        buffer_size: float = 15.0,
        out_folder: str = None
    ) -> xr.DataArray:

    # --- Load image ---
    img_ds = rasterio.open(image_file)
    img = img_ds.read().astype(np.float32)
    if img.ndim == 2:
        img = img[np.newaxis, :, :]
    bands, h_img, w_img = img.shape

    # --- Load DEM ---
    dem = rxr.open_rasterio(dem_file).squeeze()
    crs = dem.rio.crs
    dem = xr.where(dem == -9999, np.nan, dem)

    # --- Camera rotation and position ---
    R = cv2.Rodrigues(rvec)[0]
    cam_pos = (-R.T @ tvec).flatten()

    # --- Clip DEM around camera ---
    x_cam, y_cam = cam_pos[0], cam_pos[1]
    dem_clipped = dem.sel(
        x=slice(x_cam - buffer_size, x_cam + buffer_size),
        y=slice(y_cam + buffer_size, y_cam - buffer_size)
    )

    if dem_clipped.size == 0:
        raise ValueError("Clipped DEM area is empty — camera footprint does not intersect DEM.")

    # --- Create dense ortho grid at target resolution ---
    x_min, x_max = float(dem_clipped.x.min()), float(dem_clipped.x.max())
    y_min, y_max = float(dem_clipped.y.min()), float(dem_clipped.y.max())
    Nx = int(np.ceil((x_max - x_min) / target_res)) + 1
    Ny = int(np.ceil((y_max - y_min) / target_res)) + 1

    X_grid = np.linspace(x_min, x_max, Nx, dtype=np.float32)
    Y_grid = np.linspace(y_min, y_max, Ny, dtype=np.float32)
    XX, YY = np.meshgrid(X_grid, Y_grid)

    # Interpolate DEM onto ortho grid
    dem_grid = dem_clipped.interp(x=X_grid, y=Y_grid, method="nearest").values.astype(np.float32)
    ZZ = dem_grid

    # --- Flatten grid and transform to camera coordinates ---
    world_pts = np.stack([XX.ravel(), YY.ravel(), ZZ.ravel()], axis=1)
    cam_pts = (R @ world_pts.T + tvec).T

    # --- Mask points behind camera or above max elevation ---
    in_front = cam_pts[:,2] > 0
    below_max = world_pts[:,2] <= cam_pos[2] + max_elevation_above_camera
    half_fov_rad = np.radians(fov_deg / 2)
    inside_fov = np.sqrt(cam_pts[:,0]**2 + cam_pts[:,1]**2) / cam_pts[:,2] <= np.tan(half_fov_rad)
    valid = in_front & below_max & inside_fov

    cam_pts = cam_pts[valid]
    world_pts = world_pts[valid]

    # --- Project points to image coordinates ---
    img_pts, _ = cv2.fisheye.projectPoints(
        cam_pts.reshape(-1,1,3),
        rvec=np.zeros((3,1)),
        tvec=np.zeros((3,1)),
        K=K,
        D=D
    )
    u = img_pts[:,0,0]
    v = img_pts[:,0,1]

    # --- Build dense map for remap ---
    map_x = np.full(XX.shape, np.nan, dtype=np.float32)
    map_y = np.full(XX.shape, np.nan, dtype=np.float32)

    # Map valid world points back to grid indices
    ix = np.round((world_pts[:,0] - x_min) / target_res).astype(int)
    iy = np.round((world_pts[:,1] - y_min) / target_res).astype(int)
    map_x[iy, ix] = u
    map_y[iy, ix] = v

    # --- Remap image ---
    ortho = np.zeros((bands, Ny, Nx), dtype=np.float32)
    for b in range(bands):
        ortho[b] = cv2.remap(
            img[b],
            map_x,
            map_y,
            interpolation=cv2.INTER_LINEAR,
            borderMode=cv2.BORDER_CONSTANT,
            borderValue=np.nan
        )
        ortho[b][np.isnan(map_x)] = np.nan
        ortho[b][np.isnan(map_y)] = np.nan

    # --- Wrap as xarray ---
    ortho_xr = xr.DataArray(
        ortho,
        dims=("band", "y", "x"),
        coords={"x": X_grid, "y": Y_grid}
    )
    ortho_xr = ortho_xr.dropna(how='all', dim='x').dropna(how='all', dim='y')
    ortho_xr = ortho_xr.rio.write_crs(crs, inplace=False)

    # --- Optionally save ---
    if out_folder is not None:
        os.makedirs(out_folder, exist_ok=True)
        fname = os.path.basename(image_file).replace('.tiff', '_orthoimage.tiff')
        ortho_xr.rio.to_raster(os.path.join(out_folder, fname))
        print(f'Orthoimage saved: {fname}')

    return ortho_xr


os.makedirs(calib_folder, exist_ok=True)
image_files = sorted(glob(os.path.join(single_band_folder, '*.tiff')))

### CALIBRATE AND ORTHORECTIFY IN TWO GROUPS ###
# First 8 images are a different size than second 
for files, results_file in [
    [image_files[0:8], os.path.join(calib_folder, f"ch01-ch08_calibrated_cameras.csv")],
    [image_files[8:], os.path.join(calib_folder, f"ch09-ch16_calibrated_cameras.csv")]
]:

    # Calibrate cameras
    results_df = calibrate_cameras(
        image_files = files, 
        gcp_file = gcp_merged_file, 
        plot_results = False
        )
    
    # Save calibrated camera specs
    results_df.to_csv(results_file, index=False, header=True)
    print('Calibrated cameras saved to file')

    # # orthorectify
    # for i,row in tqdm(results_df.iterrows(), total=len(results_df), desc='Orthorectifying'):
    #     ortho_xr = orthorectify(
    #         image_file = row['image_file'],
    #         dem_file = refdem_file,
    #         K = row['K'],
    #         D = row['dist'],
    #         rvec = row['rvec'],
    #         tvec = row['tvec'],
    #         target_res = 0.002,
    #         out_folder = calib_folder,
    #     )

In [ ]:
df = pd.read_csv(results_file)
df['K'] = np.array(df['K'].apply(literal_eval))
df['K'][0]

## Combine calibration parameters into one file

In [ ]:
calib_files = sorted(glob(os.path.join(calib_folder, '*.csv')))
calib_list = []
for calib_file in calib_files:
    calib_list += [pd.read_csv(calib_file)]
calib = pd.concat(calib_list).reset_index(drop=True)
calib
# Evaluate values
for k in ['K', 'dist', 'K_full', 'rvec', 'tvec']:
    calib[k] = calib[k].apply(literal_eval)

out_file = os.path.join(calib_folder, "original_calibrated_cameras.csv")
calib.to_csv(out_file, index=False)
print(f'Merged calibration parameters saved to:\n{out_file}')